# An analysis of the Self-Driving Car Job Market (in the US)

The full article is available on medium: [link](https://medium.com/@jmlbeaujour/an-analysis-of-the-self-driving-car-job-market-in-the-us-dffd3b2b43ae) 


## 1. Requirements
1. indeed API in python
2. pip install beautifulsoup
3. pip install sqlite3

## 2. Sources of data
    1. indeed.com
    2. https://www.programmableweb.com/api/careerbuilder [not yet used]


## 3. Data extraction pipeline:
1. API: with the API, search for all jobs that are related to SDC. I used the following keywords: ‘self-driving car’, ‘autonomous vehicles’, ‘autonomous cars’....
2. Web-scraping: get the full description of the job. This is done using the ‘jobkey’ and the urllib library
3. The requirements for the job are extracted: education (CS, CE, Physics, …), field of study, programming skills (C++, Java, Python, etc…), area of expertise (Machine Learning, Radars, ...).
4. Outliers: we filter out the unrelated jobs postings by making sure that the full job description contains key expressions such as: “self-driving cars”, “autonomous vehicles”…
5. Manual filtering: there were still some unrelated job postings. They were removed manually.

## 4. Database to store data
For conveniency, the data is stored in a SQlite database. The structure of the database is shown below

![database](figs/db.png)


## 1. The required libraries

* Attribution: https://github.com/indeedlabs/indeed-python

In [1]:
from indeed import IndeedClient
import re
import time
import sqlite3
import glob
from bs4 import BeautifulSoup # For HTML parsing
import urllib # Website connections
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections: wait between each request
from collections import Counter # Keep track of the term counts
#from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
%matplotlib inline

## 2. Read publisher number

The publisher number is saved in a text file: "publisher_nb.txt"
Each line contains the name of the jobsite and the publisher number. Here is an example:

indeed 12345678


In [2]:
'''
read publisher number - Indeed.com
line format
indeed publisher_nbr
'''

fname = 'publisher_nbr.txt'
with open(fname, 'r') as f:
    for line in f:
        line_ls = line.split(" ")
        site_name, publisher_nbr = line_ls
        #remove newline in publisher_name
        publisher_nbr = publisher_nbr.split('\n')[0]
        #print('My publisher number for {} is: {}'.format(site_name, publisher_nbr))
f.closed

client = IndeedClient(publisher=publisher_nbr)

## Dictionaries of key words

We create multiple dictionaries of keywords for different items.

### keywords in the job description for. 

1. eduDegrees_ls: a list of degrees

2. eduField_ls: list of fields of study - It can be a single word like "physics" or composed of 2 words like "mechanical engineering".

In the case of a 2-words expression: the 1st word is a key of the dict, and the value is a list of possible 2nd word. So "software engineering" becomes {'software': ['engineering']}. If we want to include the expression "software scientist", then we just need to update the list:  {'software': [engineering, scientist]}

3. expertise_ls: list of expertise

4. progskills_ls: list of pogramming skills

5. job_requirements_ls

5. api_search_kwords : list of keywords for the API request

### MISC

1. api_job_items : list of item returned by the API for each job listing
2. db_tables_name : names of the tables in the db 
3. dbname : name of the database
4. url : generic url to scrap job description 

In [3]:
##########
# Dictionary of keywords
##########
edu_degrees = ['phd', 'ms', 'bsc', 'mba', 'master', 'masters', 'bachelors', 'bachelor', 'ba', 'bs']

edu_fields = {'cs':[], 'ce':[], 'ee':[], 'physics':[], 'electrical': ['engineering'], 'computer': ['science'],
                     'mechanical': ['engineering'], 'industrial': ['engineering']}

expertise_ls = {'machine': ['learning'], 'Deep': ['Learning'], 'computer': ['architecture'], 'Algorithms':[], 
                'GPU': ['computing'], 'parallel': ['programming'], 'Computer': ['Vision'], 
               'predictive': ['analytics'], 'vehicle':['design', 'development', 'regulations'],
               'autonomous':['vehicles', 'cars', 'vehicle', 'car'], 'robotics':[], 'radar':[], 'kinematics':[]}

progskills_ls = ['c++', 'c', 'c#', 'python', 'opencv', 'matlab', 'php', 'java', 'ruby', 'ros', 'tensorflow', 'keras']

job_requirements_ls = {'degrees': edu_degrees, 'edufields': edu_fields, 'expertise': expertise_ls,
                       'progskills': progskills_ls}
#######

#######
# OTHER PARAMETERS
#######
api_job_items = ["jobtitle", "date", "company", "country", "state", "longitude", "latitude"]
db_tables = ['jobs', 'degrees', 'progskills', 'edufields', 'expertise']
dbname = "job_listings"
url = 'https://www.indeed.com/viewjob?jk=' 
api_search_kwords = [['autonomous', 'vehicles'], ['autonomous', 'cars'], ['self-driving', 'cars'], 
                     ['self-driving', 'car'], ['self', 'driving', 'cars'], ['self driving car'], ['autonomous vehicles']]
thresh_empty = 3 #stop sending requests if 3 consecutives response are empty
#keywords = ['self-driving car']
#keywords = ['self-driving', 'car']
#keywords = ['autonomous', 'cars']
#keywords = ['autonomous vehicles']
#keywords = ['autonomous cars']
#keywords = ['autonomous-cars']
start = 0 #start at response 1
limit = 25 #Nber of job listing in respnse (simialr to number of response in page)
db_fname = 'indeed.db' #name of the database file
overwrite_db = True

In [10]:
class Indeed_JobScrap():
    '''
    This class includes methods to create the database, extract data with the API, scrap web page and 
    store all information in the database
    '''
    def __init__(self, thresh_empty, api_job_items, db_tables, db_fname, overwrite_db, 
                 job_requirements_ls):
        self.overwrite_db = overwrite_db
        self.db_fname = db_fname
        self.job_requirements_ls = job_requirements_ls
        #job items: list of data provided by the API for each job listing
        self.job_items = api_job_items
        self.listing = {}
        self.db_tables = db_tables
        self.dbname = "job_listings"
        self.url = 'https://www.indeed.com/viewjob?jk='
        #self.other_tables = ['degrees', 'progskills', 'qualifications', 'expertise']
        self.api_search_kwords = api_search_kwords
        # a tracker to keep count of how many job_responses are null - if more than self.thresh, stop sending request
        self.tracker = [] #
        self.thres = thresh_empty
    
    
    def connect_db(self):
        '''
        Create database and tables
        and connect to db
        '''
        sql_files = glob.glob('*.db')
        if (self.db_fname in sql_files) or (overwrite_db == False):
            self.conn = sqlite3.connect(self.db_fname)
            self.c = self.conn.cursor()
        else:
            print('Create table')
            #open connection to local file
            self.conn = sqlite3.connect(self.db_fname)
            self.c = self.conn.cursor()
            # Create tables
            for this_table in self.db_tables:
                if this_table == 'jobs':
                    sql_create_table="CREATE TABLE {} (jobkey, jobtitle, company, date, country, state, longitude, latitude)".format(this_table)
                else:
                    col_name = this_table + '_req'
                    sql_create_table = "CREATE TABLE {} (jobkey, {})".format(this_table, col_name)
                
                try:
                    self.c.execute(sql_create_table)
                    print('Created Table: {} in database {}'.format(this_table, self.dbname) )
                    self.conn.commit()
                except:
                    print('Unable to create table {}'.format(this_table))


    def jobkey_in_db(self, jk):
        '''
        check if jobkey in database
        return True/False
        '''#check if job in db
        flag_jobindb = False
        flag_error = False
        this_table = self.db_tables[0]
        sql_select = "SELECT {}.jobkey FROM {} WHERE {}.jobkey = '{}';".format(this_table, this_table, this_table, jk)
        try:
            self.c.execute(sql_select)
            result = self.c.fetchall()
            if len(result) != 0:
                flag_jobindb = True
        except:
            # Insert a row of data
            print('Unable to execute SQL query: SELECT')
            flag_error = True
        print('job entry already recorded?: {}'.format(flag_jobindb))
        return flag_jobindb
        
    
    def insert_job_entry(self, job_entry):
        '''
        insert job_entry to database
        '''
        for this_table in self.db_tables:
            if this_table == 'jobs':
                sql_insert = "INSERT INTO {} VALUES ('{}','{}', '{}', '{}', '{}', '{}', '{}', '{}');".format(this_table, job_entry['jobkey'], job_entry['jobtitle'], job_entry['company'], job_entry['date'], job_entry['country'], job_entry['state'], job_entry['longitude'], job_entry['latitude'])
                try:
                    self.c.execute(sql_insert)
                    self.conn.commit()
                except:
                    print('Unable to insert this entry in  {}'.format(self.table_name))
            else:
                for item in job_entry[this_table]:
                    sql_insert = "INSERT INTO {} VALUES ('{}','{}');".format(this_table, job_entry['jobkey'], item)
                
                    try:
                        self.c.execute(sql_insert)
                        self.conn.commit()
                    except:
                        print('Unable to save item {} in {}'.format(item, this_table))
    
    
    
    def update_job_entry(self, job_entry):
        '''
        insert job_entry to database
        '''
        for this_table in self.db_tables:
            if this_table == 'jobs':
                sql_update = "UPDATE {} SET {}='{}', {}='{}', {}='{}', {}='{}', {}='{}', {}='{}', {}='{}' WHERE jobkey='{}';".format(this_table, 
            'jobtitle', job_entry['jobtitle'], 'company', job_entry['company'], 'date', job_entry['date'], 
            'country', job_entry['country'], 'state', job_entry['state'], 'longitude', 
            job_entry['longitude'], 'latitude', job_entry['latitude'], job_entry['jobkey'] )
                self.c.execute(sql_update)
                self.conn.commit()
                try:
                    self.c.execute(sql_update)
                    self.conn.commit()
                except:
                    print('Unable to update this entry in  {}'.format(this_table))
            else:
                for item in job_entry[this_table]:
                    col_name = this_table+'_req'
                    sql_update = "UPDATE {} SET {}='{}' WHERE jobkey='{}';".format(this_table, col_name, item, this_table, job_entry['jobkey'])
                    try:
                        self.c.execute(sql_update)
                        self.conn.commit()
                    except:
                        print('Unable to update item {} in {}'.format(item, this_table))
            
            
            
    def push2db(self, job_entry):
        '''
        Save all dat of job_entry to database: 
        jobkey, jobtitle, company, data, country, state, longitude, latitude,
        degrees, qualifications, expertise, progskills
        '''
        jk = job_entry['jobkey']
        flag_jobindb = self.jobkey_in_db(jk)
        
        if flag_jobindb == False:
            self.insert_job_entry(job_entry)
        else:
            self.update_job_entry(job_entry)

            
    ##############
    # Text scrapping utils
    ##################
    def sanitize_text(self, text):
        try:
            text = text.decode('unicode_escape').encode('ascii', 'ignore') 
        except:
            pass

        #Remove special characters
        special_chars = "!@#$%^&*()[]{};:,./<>?\|`~=_"
        for special_char in special_chars:
            clean_text = text.replace(special_char, " ")
        #remove newline
        clean_text = " ".join(clean_text.split())
        clean_text = re.sub("[^a-zA-Z+2]"," ", clean_text)
        return clean_text
    
    
    
    def get_jobsListing(self, keywords, start, limit):
        query = ""
        if len(keywords) > 0:
            #create query similar to "autonomous+cars"
            query = "+".join(keywords)
            params = {
                      'q' : query,
                      # 'l' : "austin",
                      'start': start,
                      'limit': limit, #maximum nbr of results returned
                      'userip' : "1.2.3.4",
                      'useragent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2)"
                     }

            return client.search(**params)
        else:
            return []                                                                                                            
    
                                                                                                                                                                                                             
    def snif_jobDescription(self, keywords, jobkey):
        this_url = self.url+str(jobkey)
        flag_error = False
        try:
            site = urllib.request.urlopen(this_url).read() # Connect to the job posting
            soup_obj = BeautifulSoup(site, "lxml")
        except:
            print('page not found')
            flag_error = True
        
        if flag_error == False:
            # search for all <span tag
            spans = soup_obj.find_all('span', attrs={'id':"job_summary"})
            if len(spans) == 1:
                #convert soup object to text
                text = spans[0].get_text()
                #Cleaning text
                clean_text = self.sanitize_text(text)

                #if clean_text contains any of the sdc_expressions
                kword = " ".join(keywords)
                if ( kword in clean_text ):
                    text_split = clean_text.split()
                    return text_split
                else:
                    return []
            else:
                return []


    def extract_jobRequirements(self, keywords, jobkey):
        #get text job description
        job_description_txt = self.snif_jobDescription(keywords, jobkey)
        #save job requirements in job_req
        job_req = {}
        for i in self.job_requirements_ls:
            job_req[i] = []

        #better to scroll thru text once
        for idx, word in enumerate(job_description_txt):
            w_lower = word.lower()
            
            for key in self.job_requirements_ls:
                if key == 'expertise' or key == 'edufields':
                    if w_lower in self.job_requirements_ls[key]:
                        if len(self.job_requirements_ls[key][w_lower])==0:
                            #the word has no child in list of job req  (like 'cs':[]) 
                            if len(job_req[key])==0:
                                job_req[key] = [w_lower]
                            elif w_lower not in job_req[key]:
                                job_req[key].append(w_lower)
                        else:
                            #check the word is not the last of the job_description (otherwise next_word does not exist)
                            if idx < len(job_description_txt)-1:
                                next_word = job_description_txt[idx+1].lower()
                                #check if expression=word+next_word is in our list of job_requirements of that key
                                if next_word in self.job_requirements_ls[key][w_lower]:
                                    expression = w_lower+" "+next_word
                                    if len(job_req[key])==0:
                                        job_req[key] = [expression]
                                    elif expression not in job_req[key]:
                                        job_req[key].append(expression)
                                
                else:
                    if w_lower in self.job_requirements_ls[key]:
                        if len(job_req[key])==0:
                            job_req[key] = [w_lower]
                        elif w_lower not in job_req[key]:
                            job_req[key].append(w_lower)
        
        return job_req

                            
    def record(self, keywords, start, limit):
        while len(self.tracker) < self.thres:
            #get query responses from API (pagination)
            query_response = self.get_jobsListing(keywords, start, limit)
            cnt_entry = 0
            print('**************', start)
            for job_entry in query_response['results']:
                job_details = {}
                cnt_entry += 1
                jobkey = job_entry['jobkey']
                job_details['jobkey'] = jobkey
                print('Job Entry # {} [{}]'.format(cnt_entry, jobkey))
                #Make a query on this jobkey
                job_entry_response = client.jobs(jobkeys = (jobkey, ))
                if len(job_entry_response['results']) > 0:
                    #reset tracker
                    self.tracker = []
                    job_entry_response = job_entry_response['results'][0]
                    ##save job specs (company, title etc...)  ########
                    for job_item in self.job_items:
                        if job_item in job_entry_response:
                            job_details[job_item] = job_entry_response[job_item]
                        else:
                            job_details[job_item] = ""
                    
                    #extract job requirements from job description
                    job_req = self.extract_jobRequirements(keywords, jobkey)
                    job_specs = {**job_details, **job_req}
                    #save job entry in db
                    self.push2db(job_specs)       
                    time.sleep(1)
                    print('Jobkey {}'.format(jobkey))
                    print()

                else:
                    self.tracker.append(1)
            
            #check consistency n_entry and limit
            start = start + limit

    
    def remove_outliers(self, outlier_adress):
        #Open outlier file
        ls_outliers = []
        flag_err = True
        with open(outlier_adress) as f:
            lines = f.readlines()
            for line in lines:
                line = line[0:-1]
                ls_outliers.append(line)
            flag_err = False

        if flag_err==False:
            print('************ Files of outliers found and open')
            for jk in ls_outliers:
                print("********** outlier: {}".format(jk))
                #check that the outlier is in database main table: jobs 
                sql_select = "SELECT jobkey FROM jobs WHERE jobkey='{}';".format(jk) 
                try:
                    self.c.execute(sql_select)
                    result = self.c.fetchall()
                    flag_err = False
                except:
                    print("cannot check if table contains outlier obkey")
                    flag_err = True

                if flag_err == False and len(result)>1:
                    #Delete the entry in all tables
                    for this_table in self.db_tables:
                        sql_del =  "DELETE FROM {} WHERE jobkey='{}';".format(this_table, jk)
                        try:
                            self.c.execute(sql_delete)
                            self.conn.commit()
                        except:
                            print('could not delete {} from table: jobs')
                        print('Entry {} deleted'.format(jk))
        print('Process finished........................')

In [11]:
jobsScrap = Indeed_JobScrap(thresh_empty, 
                            api_job_items, 
                            db_tables, 
                            db_fname, 
                            overwrite_db,
                            job_requirements_ls)
#connect to database
jobsScrap.connect_db()
for search_kword in api_search_kwords:
    jobsScrap.record(search_kword, start, limit)
    


************** 0
Job Entry # 1 [c26251ab6813a6ee]
job entry already recorded?: True
Jobkey c26251ab6813a6ee

Job Entry # 2 [2f000f5fb372b471]
job entry already recorded?: True
Jobkey 2f000f5fb372b471

Job Entry # 3 [0aba2b5d5cba3b6b]
job entry already recorded?: True
Jobkey 0aba2b5d5cba3b6b

Job Entry # 4 [477920bea36b9157]
job entry already recorded?: True
Jobkey 477920bea36b9157

Job Entry # 5 [c72f9233c22e5bf7]
job entry already recorded?: True
Jobkey c72f9233c22e5bf7

Job Entry # 6 [6107b741cadd76df]
job entry already recorded?: True
Jobkey 6107b741cadd76df

Job Entry # 7 [201d18049accbe3c]
job entry already recorded?: True
Jobkey 201d18049accbe3c

Job Entry # 8 [e511e6d451fff5d1]
job entry already recorded?: True


KeyboardInterrupt: 

# Select manually some outliers
This part of the pipeline needs to be improved.
Using DBBrowser for SQLite, I order the jobs by job title and search for any job title that is not related to SDC.
I save all the jobkeys of the "mistakes" in a text file called "outlier.txt".

Below, we automatically scroll through all entries in database and delete keyjobs that are outliers.

In [ ]:
outlier_adress = 'outliers.txt'
jobsScrap.remove_outliers(outlier_adress)